In [1]:
import sys

import oct2py
import numpy as np
from numpy.testing import assert_allclose
from skimage.filters import threshold_otsu
import matplotlib.pyplot as plt
import scipy.ndimage

oc = oct2py.Oct2Py()
oc.pkg('load', 'image')

sys.path.append('../lccd-python/')
import blob_detector
import test_utils

In [2]:
# Test blob_detector.matlab_style_gauss2D
filtersize1 = 100
filtersize2 = 4
sigma = 1.25
flsize = 7
assert_allclose(blob_detector.matlab_style_gauss2D((flsize, flsize), sigma),
                oc.fspecial('gaussian', flsize, sigma)
               )

In [3]:
# Test blob_detector.im2bw
I = np.random.uniform(0, 4, size=(32, 32))
level = np.random.uniform(0, 1)

assert_allclose(
    blob_detector.im2bw(I, level),
    oc.im2bw(I, level)
)

In [4]:
# Test blob_detector.filter_closed_regions_by_area
arr = np.zeros((256, 256), dtype='int64')
arr[10: 20, 1: 2] = 1 # area = 10
arr[30: 40, 3: 6] = 2 # area = 30
arr[50: 60, 7: 17] = 3 # area = 100

ca = np.zeros((256, 256), dtype='int64')
ca[30: 40, 3: 6] = 2


res = blob_detector.filter_closed_regions_by_area(arr, 20, 50)
assert_allclose(ca, res)

In [31]:
X = 64
Y = 64
T = 1000
min_area, max_area = 20, 100
bd = blob_detector.BlobDetector(filtersize1, filtersize2, sigma, flsize, min_area, max_area, matlab_conv=True, debug=True)
tv = np.random.uniform(0, 1, size=(X, Y, T)) + 1e-8 # tv in R^(XY x T) _+
python_res = bd.apply(tv)

In [32]:
mavefil1 = oc.ones(1, {filtersize1}) / filtersize1
mavefil2 = oc.ones(1, {filtersize2}) / filtersize2
tv = tv.reshape(-1, T)
smooth_tv = oc.max(oc.conv2(tv, mavefil2, 'same') - oc.conv2(tv, mavefil1, 'same'), 0)
assert_allclose(python_res["smooth_tv"], smooth_tv)

max_lumi_map = oc.max(smooth_tv, [], 2)
assert_allclose(max_lumi_map.flatten(), python_res["max_lumi_map"].flatten())

sp_fil = oc.fspecial('gaussian', flsize, sigma) - oc.fspecial('average', flsize)
assert_allclose(sp_fil, bd.sp_fil)

lumi_map = oc.max(oc.conv2(oc.reshape(max_lumi_map, X, Y), sp_fil, 'valid'), 0)
try:
    assert_allclose(python_res["lumi_map"], lumi_map) # ASSERT!
except Exception as e:
    print(e)
    lumi_map = python_res["lumi_map"]


Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 2480 / 3364 (73.7%)
Max absolute difference: 0.0350906
Max relative difference: 698.59307228
 x: array([[0.016457, 0.013094, 0.      , ..., 0.010105, 0.014736, 0.003477],
       [0.006801, 0.004134, 0.      , ..., 0.      , 0.000548, 0.      ],
       [0.004413, 0.00074 , 0.      , ..., 0.      , 0.      , 0.      ],...
 y: array([[0.016457, 0.006801, 0.004413, ..., 0.004597, 0.005382, 0.      ],
       [0.013094, 0.004134, 0.00074 , ..., 0.004558, 0.005749, 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.005517, 0.002848],...


In [33]:
cont_enhanced_map = python_res["cont_enhanced_map"]

In [34]:
threshold = oc.graythresh(cont_enhanced_map)
try:
    assert_allclose(python_res["threshold"], threshold)
except Exception as e:
    print(e)
    threshold = python_res["threshold"]


Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 1 / 1 (100%)
Max absolute difference: 0.0009421
Max relative difference: 0.00363991
 x: array(0.259766)
 y: array(0.258824)


In [35]:
bwm = oc.im2bw(cont_enhanced_map, threshold)
assert_allclose(python_res["bwm"], bwm)

In [36]:
test_utils.same_labels(python_res["label_mat"], oc.bwlabel(bwm, 8))

In [37]:
label_mat2 = python_res["label_mat2"]